EULERIANCYCLE(Graph)

        form a cycle Cycle by randomly walking in Graph (don't visit the same edge twice!)

        while there are unexplored edges in Graph

            select a node newStart in Cycle with still unexplored edges

            form Cycle’ by traversing Cycle (starting at newStart) and then randomly walking

            Cycle ← Cycle’
            
        return Cycle

Given: An Eulerian directed graph, in the form of an adjacency list.

Return: An Eulerian cycle in this graph.

Euler’s Theorem: Every balanced, strongly connected directed graph is Eulerian.

In [2]:
def FormatGraph(graph):
  graph_dict = {}
  for i in range(len(graph)):
    key_adjacent_nodes = graph[i].split(' -> ')
    graph_dict.setdefault(int(key_adjacent_nodes[0]),[])
    for adjacent_node in key_adjacent_nodes[1].split(','):
      graph_dict[int(key_adjacent_nodes[0])].append(int(adjacent_node))
  return graph_dict

In [3]:
def VisitedEdges(graph_dict):
  visited_edges_dict = {}
  for key,adjacent_nodes in graph_dict.items():
    for node in adjacent_nodes:
      visited_edges_dict.setdefault((key,node),[])
      visited_edges_dict[(key,node)].append(0)
  return visited_edges_dict

In [4]:
def NodeInDegree(graph_dict,node):
  node_in_degree = 0
  for values_list in graph_dict.values():
    if node in values_list:
      node_in_degree = node_in_degree + 1
  return node_in_degree

In [5]:
def NodeOutDegree(graph_dict,node):
  if node in graph_dict.keys():
    return len(graph_dict[node])
  else: #node has no adjacent nodes
    return 0

It may not be obvious, but a good implementation of EULERIANCYCLE will work in
linear time. To achieve this runtime speedup, you would need to use an efficient data
structure in order to maintain the current cycle that Leo is building as well the list of
unused edges incident to each node and the list of nodes on the current cycle that have
unused edges.

Lookups in lists are O(n), lookups in dictionaries are amortized O(1), with regard to the number of items in the data structure. In Python, the average time complexity of a dictionary key lookup is O(1), since they are implemented as hash tables. The time complexity of lookup in a list is O(n) on average.

In [6]:
from random import randint,randrange

In [7]:
def VisitedEdgesDictSum(visited_edges_dict):
  visited_edges_dict_sum = 0
  for values_list in visited_edges_dict.values():
    visited_edges_dict_sum = visited_edges_dict_sum + sum(values_list)
  return visited_edges_dict_sum

In [8]:
def VisitedEdgesDictValueCount(visited_edges_dict):
  visited_edges_dict_value_count = 0
  for values_list in visited_edges_dict.values():
    visited_edges_dict_value_count = visited_edges_dict_value_count + len(values_list)
  return visited_edges_dict_value_count

In [9]:
def EulerianCycle(graph_dict):
  visited_edges_dict = VisitedEdges(graph_dict)
  starting_node = randint(min(graph_dict.keys()),max(graph_dict.keys())) #lower and upper bound included
  cycle = [starting_node]
  visited_edges_dict_value_count = VisitedEdgesDictValueCount(visited_edges_dict)
  while VisitedEdgesDictSum(visited_edges_dict) < visited_edges_dict_value_count: #repeat until Eulerian cycle is found --> input is an Eulerian directed graph --> Eulerian cycle can always be found
    #while loop entered --> sum(visited_edges_dict.values()) < len(visited_edges_dict) --> cycle smaller than Eulerian cycle is being formed
    possible_adjacent_nodes = [key[1] for key in visited_edges_dict.keys() if key[0] == cycle[len(cycle)-1] and visited_edges_dict[key].count(0) > 0]
    if len(possible_adjacent_nodes) == 0 and VisitedEdgesDictSum(visited_edges_dict) < visited_edges_dict_value_count: #cycle smaller than Eulerian cycle completed as we got stuck at starting node --> all edges are not visited
      #possible_starting_nodes_list = [node for node in cycle if NodeOutDegree(graph_dict,node) >= NodeOutDegree(graph_dict,cycle[0])] #no, this way we are choosing node regardless of the number of times it appeared in cycle
      #possible_starting_nodes_list = [visited_edge[0] for node in cycle for visited_edge in visited_edges_dict.keys() if visited_edge[0] == node and visited_edges_dict[visited_edge] == 1] --> this caused efficiency problems
      possible_starting_nodes_list = [node for node in cycle if NodeOutDegree(graph_dict,node) > cycle.count(node)] #if NodeOutDegree(node) > number of times node occurs in cycle then there are unused outgoing edges (and unused ingoing edges because of unused outgoing edges, unused outgoing edges must be visited by unused incoming edges), every occurence means that one outgoing edge is used
      if len(possible_starting_nodes_list) == 0:
        for key in visited_edges_dict.keys():
          if sum(visited_edges_dict[key]) == 0:
            print(key)
      if len(possible_starting_nodes_list) == 1:
        starting_node = possible_starting_nodes_list[0] #choose only available starting node
      else:
        starting_node = possible_starting_nodes_list[randrange(0,len(possible_starting_nodes_list))] #randomly choose new starting node among nodes with higher NodeOutDegree than previous starting node
      cycle = cycle[cycle.index(starting_node):len(cycle)] + cycle[1:cycle.index(starting_node)+1] #construct new_cycle using previous cycle
    else: #len(possible_adjacent_nodes) > 1 and sum(visited_edges_dict.values()) < len(visited_edges_dict) --> cycle is not finished yet
      if len(possible_adjacent_nodes) == 1:
        next_node = possible_adjacent_nodes[0]
        #visited_edges_dict[(cycle[len(cycle)-1], next_node)] = 1
        visited_edges_dict_value_list = visited_edges_dict[(cycle[len(cycle)-1], next_node)]
        visited_edges_dict_value_list[visited_edges_dict_value_list.index(0)] = 1 #always update the first zero
        cycle.append(next_node)
      else:
        next_node = possible_adjacent_nodes[randint(0,len(possible_adjacent_nodes)-1)] #lower and upper bound included
        visited_edges_dict_value_list = visited_edges_dict[(cycle[len(cycle)-1], next_node)]
        visited_edges_dict_value_list[visited_edges_dict_value_list.index(0)] = 1 #always update the first zero
        cycle.append(next_node)
  return cycle

In [10]:
def PrintResult(eulerian_cycle):
  string_to_print = ''
  for node in eulerian_cycle:
    string_to_print = string_to_print + str(node) + '->'
  print(string_to_print[0:len(string_to_print)-2])

In [11]:
def PrintResultToFile(eulerian_cycle):
  f = open("task_result.txt","w")
  string_to_print = ''
  for node in eulerian_cycle:
    string_to_print = string_to_print + str(node) + '->'
  f.write(string_to_print[0:len(string_to_print)-2])
  f.close()

In [ ]:
graph = [
'0 -> 3',
'1 -> 0',
'2 -> 1,6',
'3 -> 2',
'4 -> 2',
'5 -> 4',
'6 -> 5,8',
'7 -> 9',
'8 -> 7',
'9 -> 6'
]

In [ ]:
graph_dict = FormatGraph(graph)
graph_dict

{0: [3],
 1: [0],
 2: [1, 6],
 3: [2],
 4: [2],
 5: [4],
 6: [5, 8],
 7: [9],
 8: [7],
 9: [6]}

In [ ]:
PrintResult(EulerianCycle(graph_dict))

6->8->7->9->6->5->4->2->1->0->3->2->6


In [14]:
with open('/content/rosalind_ba3f.txt') as task_file:
  graph = [line.rstrip() for line in task_file]

In [15]:
graph_dict = FormatGraph(graph)

In [16]:
PrintResultToFile(EulerianCycle(graph_dict))